# Сборка семантического графа

#### Импорт библиотек

In [ ]:
import geopandas as gpd
import networkx as nx
from pathlib import Path

from sloyka import Semgraph

package_dir = str(Path.cwd().parent)

#### Инициализация класса семантического графа

In [ ]:
sm = Semgraph()

#### Загрузка данных
###### Рекомендуется использовать после класса Geocoder

In [3]:
data_path = package_dir + "/sloyka/sample_data/processed/"

In [4]:
test_gdf  = gpd.read_file(data_path + "-10696.geojson")

#### Построение семантического графа

In [5]:
G = sm.build_graph(test_gdf[:len(test_gdf)//2],
                   id_column='id',
                   text_column='text',
                   text_type_column='type',
                   toponym_column='only_full_street_name_numbers',
                   toponym_name_column='initial_street',
                   toponym_type_column='Toponims',
                   post_id_column='post_id',
                   parents_stack_column='parents_stack',
                   location_column='Location',
                   geometry_column='geometry')

Extracting keywords from post chains...


100%|██████████| 181/181 [00:38<00:00,  4.68it/s]


Extracting keywords from comment chains...


100%|██████████| 25/25 [00:01<00:00, 21.48it/s]


Extracting keywords from replies...


100%|██████████| 19/19 [00:01<00:00, 18.88it/s]


Calculating semantic closeness...


100%|██████████| 13861/13861 [00:17<00:00, 805.47it/s]


###### Получаем количество узлов графа

In [ ]:
len(G.nodes)

###### Обновляем граф

In [ ]:
G = sm.update_graph(G,
                    test_gdf[len(test_gdf)//2:],
                    id_column='id',
                    text_column='text',
                    text_type_column='type',
                    toponym_column='only_full_street_name_numbers',
                    toponym_name_column='initial_street',
                    toponym_type_column='Toponims',
                    post_id_column='post_id',
                    parents_stack_column='parents_stack',
                    location_column='Location',
                    geometry_column='geometry')

###### Проверяем что количество узлов стало больше

In [ ]:
len(G.nodes)

###### Смотрим атрибуты узла с номером дома в id узла

In [ ]:
G.nodes['набережная канала Грибоедова 108']

###### Добавляем в атрибуты узлов со зданиями атрибут с текстом, из которого был извлечён топоним

In [10]:
G = sm.get_house_text_id(G, test_gdf, 'id', 'text')

###### Проверяем, что атрибут добавился

In [ ]:
G.nodes['набережная канала Грибоедова 108']

###### Смотрим количество связей в графе

In [ ]:
len(G.edges)

###### Добавляем связи расстояния между топонимами в граф

In [ ]:
G = sm.calculate_distances(G)

###### Проверяем, что в граф добавились новые связи

In [ ]:
len(G.edges)

###### Загружаем слои территориально-административного деления города

In [15]:
districts = gpd.read_file(data_path + "districts.geojson")
mo = gpd.read_file(data_path + "municipalities.geojson")

###### Смотрим количество узлов в графе

In [ ]:
len(G.nodes)

###### Добавляем узлы районов и муниципалитетов в граф

In [17]:
G = sm.add_city_graph(G, districts, mo, 'city_id', 'district_name', 'name', 'geometry')

###### Проверяем, что в графе появились новые узлы

In [ ]:
len(G.nodes)

###### Сохранение полученного графа в формате ``graphml``

In [ ]:
# nx.write_graphml(G, "semantic_graph_example.graphml", encoding = 'utf-8')